In [10]:
import pandas as pd

file_path = "/Users/alexanderprinz/Downloads/financials.feather"
findata = pd.read_feather(file_path)
print(findata.info())
print(findata.head(20))

display(findata)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2438957 entries, 0 to 2438956
Columns: 135 entries, index to bvd_table
dtypes: float64(87), int64(1), object(47)
memory usage: 2.5+ GB
None
    index          idnr currency nat_prim_code nace_prim_code naics_core_code  \
0       0  AT9130090333      EUR        P85320           8532            6111   
1       1  AT9010000033      EUR        L68209           6820            5311   
2       2  AT9010000033      EUR        L68209           6820            5311   
3       3  AT9010000033      EUR        L68209           6820            5311   
4       4  AT9010000033      EUR        L68209           6820            5311   
5       5  AT9010000033      EUR        L68209           6820            5311   
6       6  AT9010000033      EUR        L68209           6820            5311   
7       7  AT9010000033      EUR        L68209           6820            5311   
8       8  AT9010000033      EUR        L68209           6820            5311   


,index,idnr,currency,nat_prim_code,nace_prim_code,naics_core_code,ussic_core_code,sd_isin,sd_sedol,sd_ticker,...,solr,gear,ppe,tpe,sct,ace,sfpe,wcpe,tape,bvd_table
0,0,AT9130090333,EUR,P85320,8532,6111,821,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,amadeus_m
1,1,AT9010000033,EUR,L68209,6820,5311,651,None,None,None,...,67.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,amadeus_m
2,2,AT9010000033,EUR,L68209,6820,5311,651,None,None,None,...,67.99,NaN,NaN,NaN,NaN,NaN,6322.0,NaN,9298.0,amadeus_m
3,3,AT9010000033,EUR,L68209,6820,5311,651,None,None,None,...,76.45,NaN,NaN,NaN,NaN,NaN,5671.0,NaN,7419.0,amadeus_m
4,4,AT9010000033,EUR,L68209,6820,5311,651,None,None,None,...,80.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,amadeus_m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2438952,94938,DE8350148522,EUR,C10000,1000,311,200,None,None,None,...,89.02,5.69,223.0,NaN,NaN,48.0,611.0,13.0,687.0,amadeus_v
2438953,94939,DE8350148522,EUR,C10000,1000,311,200,None,None,None,...,88.37,4.86,287.0,704.0,6.82,48.0,721.0,11.0,816.0,amadeus_v
2438954,94940,DE8350148522,EUR,C10000,1000,311,200,None,None,None,...,78.71,16.55,341.0,795.0,6.09,48.0,352.0,21.0,448.0,amadeus_v
2438955,94941,DE8350148522,EUR,C10000,1000,311,200,None,None,None,...,81.36,16.02,406.0,919.0,5.53,51.0,610.0,37.0,750.0,amadeus_v


In [12]:
reviews = pd.read_csv("reviews.csv")
unique_company_IDs_20K = reviews['company_uuid'].unique()

In [13]:
findata["company_uuid"]

KeyError: 'company_uuid'

In [20]:
columns_to_correlate = findata.iloc[:, 56:-1]
display(columns_to_correlate)

,fias,ifas,tfas,ofas,cuas,stok,debt,ocas,cash,toas,...,shlq,solr,gear,ppe,tpe,sct,ace,sfpe,wcpe,tape
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8248509.0,64987.0,6393482.0,1790040.0,993076.0,0.0,0.0,993076.0,295860.0,9241586.0,...,2.08,67.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8096843.0,43325.0,6263478.0,1790040.0,1201506.0,0.0,0.0,1201506.0,631984.0,9298349.0,...,2.12,67.99,NaN,NaN,NaN,NaN,NaN,6322.0,NaN,9298.0
3,6663558.0,0.0,5130694.0,1532864.0,755000.0,0.0,0.0,755000.0,183739.0,7418557.0,...,3.25,76.45,NaN,NaN,NaN,NaN,NaN,5671.0,NaN,7419.0
4,6545364.0,86650.0,4904674.0,1554040.0,904014.0,0.0,0.0,904014.0,231555.0,7449378.0,...,4.00,80.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2438952,34003895.0,2960.0,4870068.0,29130866.0,15424875.0,936234.0,0.0,14488641.0,6370839.0,49428770.0,...,17.58,89.02,5.69,223.0,NaN,NaN,48.0,611.0,13.0,687.0
2438953,38514154.0,5541.0,5069816.0,33438797.0,30808150.0,976405.0,1741017.0,28090728.0,20078009.0,69322304.0,...,20.60,88.37,4.86,287.0,704.0,6.82,48.0,721.0,11.0,816.0
2438954,25528939.0,98263.0,6829689.0,18600987.0,21032711.0,1366753.0,2619266.0,17046692.0,15864343.0,46561650.0,...,6.04,78.71,16.55,341.0,795.0,6.09,48.0,352.0,21.0,448.0
2438955,53395718.0,55864.0,10484064.0,42855790.0,32813897.0,1436115.0,3413019.0,27964763.0,26573465.0,86209615.0,...,6.24,81.36,16.02,406.0,919.0,5.53,51.0,610.0,37.0,750.0
